### 포맷 다른 데이터 병합

(1) 참조 데이터가 필요없는 경우

In [3]:
import os
import pandas as pd
os.chdir(r"C:\Users\Gyu\Desktop\mywork\FastOnline\Part 4. 머신러닝을 위한 필수 전처리\데이터")

In [5]:
df1 = pd.read_csv("날짜포맷이다른데이터1.csv")
df2 = pd.read_csv("날짜포맷이다른데이터2.csv")
df1.head()

,날짜,V1
0,2018-01-01,0.330646
1,2018-01-02,0.418242
2,2018-01-03,0.307254
3,2018-01-04,0.441689
4,2018-01-05,0.402440


In [6]:
def date_type_converter(value):
    YYYY, MM, DD = value.split('-')
    return YYYY + "년 " + str(int(MM)) + "월 " + str(int(DD)) + "일"

df1['날짜'] = df1['날짜'].apply(date_type_converter)
df1.head()

,날짜,V1
0,2018년 1월 1일,0.330646
1,2018년 1월 2일,0.418242
2,2018년 1월 3일,0.307254
3,2018년 1월 4일,0.441689
4,2018년 1월 5일,0.402440


In [7]:
merged_df = pd.merge(df1, df2, on = '날짜')
merged_df.head()

,날짜,V1,V2
0,2018년 1월 1일,0.330646,0.936350
1,2018년 1월 2일,0.418242,0.658967
2,2018년 1월 3일,0.307254,0.895686
3,2018년 1월 4일,0.441689,0.047194
4,2018년 1월 5일,0.402440,0.520925


(2) 참조 데이터가 필요한 경우

In [8]:
ref_df = pd.read_csv("시승격정보.csv", encoding = "cp949")
df1 = pd.read_excel("인구수데이터.xlsx", sheet_name = "202006")
df2 = pd.read_excel("인구수데이터.xlsx", sheet_name = "201108")

# ref_df를 사전으로 변환
ref_dict = ref_df.set_index('승격전')['승격후'].to_dict() # 인덱스 설정 및 시리즈로 변환
ref_dict

{'여주군': '여주시', '당진군': '당진시'}

In [9]:
# ref_dict을 이용하여 키 변수 값 변환
df2['행정구역'] = df2['행정구역'].replace(ref_dict)
df2.head()

,행정구역,총인구수,남자인구수,여자인구수
0,여주시,109019,55122,53897
1,아산시,271356,138150,133206
2,당진시,148175,76555,71620


In [10]:
# df1과 df2에 있는 변수명이 모두 같으므로, 변수명을 수정해야 함
df1.set_index("행정구역", inplace = True) # 행정 구역 변수명은 바꾸지 않기 위해, 인덱스로 설정
df2.set_index("행정구역", inplace = True) # 행정 구역 변수명은 바꾸지 않기 위해, 인덱스로 설정

df1 = df1.add_prefix("202006_")
df2 = df2.add_prefix("201108_")
merged_df = pd.merge(df1, df2, left_index = True, right_index = True)
merged_df.head()

,202006_총인구수,202006_남자인구수,202006_여자인구수,201108_총인구수,201108_남자인구수,201108_여자인구수
행정구역,,,,,,
여주시,111341,56582,54759,109019,55122,53897
당진시,166084,88015,78069,148175,76555,71620
아산시,313054,162398,150656,271356,138150,133206


### 거리 기반 데이터 병합

In [11]:
df1 = pd.read_csv("2019년_서울_아파트매매_실거래가.csv", encoding = "cp949")
df2 = pd.read_csv("2019년_서울시_아파트주소.csv", encoding = "cp949")
df = pd.merge(df1, df2,left_on = ['법정동', '도로명', '아파트'], right_on = ['읍면동명', '도로명', '건물명'])
df.head()

,거래일자,법정동,도로명,아파트,층,전용면적,건축년도,거래금액,시군구명,읍면동명,건물명,경도,위도
0,2019-10-23,면목동,겸재로,동서그랜드맨션,4,131.34,1993,43000,중랑구,면목동,동서그랜드맨션,127.075856,37.588817
1,2019-10-25,면목동,면목로,킴스아파트,4,38.91,2018,28900,중랑구,면목동,킴스아파트,127.084734,37.592972
2,2019-10-29,면목동,용마산로86길,면목동2차미소지움아파트,12,78.56,2005,45400,중랑구,면목동,면목동2차미소지움아파트,127.094170,37.589641
3,2019-12-04,면목동,용마산로86길,면목동2차미소지움아파트,4,81.96,2005,45500,중랑구,면목동,면목동2차미소지움아파트,127.094170,37.589641
4,2019-12-26,면목동,용마산로86길,면목동2차미소지움아파트,3,82.28,2005,45500,중랑구,면목동,면목동2차미소지움아파트,127.094170,37.589641


In [12]:
df3 = pd.read_excel("지하철역_위경도.xlsx")
# 거리 행렬 생성을 위한 컬럼 추출
df_location = df[['경도', '위도']]
df3_location = df3[['경도', '위도']]

In [15]:
# !pip install haversine # 위경도 거리 계산을 위한 모듈 설치

You should consider upgrading via the 'c:\users\gyu\anaconda3\envs\py37\python.exe -m pip install --upgrade pip' command.


In [16]:
# 거리 행렬 생성
from scipy.spatial.distance import cdist
from haversine import haversine

dist_mat = cdist(df_location, df3_location, metric = haversine)
# 'euclidean', 'cityblock', 'jaccard'

In [17]:
close_subway_index = dist_mat.argsort()[:, 0] # axis = 0, 최소 index 반환
df['가까운역'] = df3.iloc[close_subway_index]['역명'].values 
# 새로운 시리즈를 만들 때는 list, ndarray를 사용하는 것이 바람직하다!

df['가까운역까지_거리'] = dist_mat[close_subway_index][:, 0]

df.head()

,거래일자,법정동,도로명,아파트,층,전용면적,건축년도,거래금액,시군구명,읍면동명,건물명,경도,위도,가까운역,가까운역까지_거리
0,2019-10-23,면목동,겸재로,동서그랜드맨션,4,131.34,1993,43000,중랑구,면목동,동서그랜드맨션,127.075856,37.588817,중랑역[경춘선],22.308503
1,2019-10-25,면목동,면목로,킴스아파트,4,38.91,2018,28900,중랑구,면목동,킴스아파트,127.084734,37.592972,상봉역[7호선],25.366110
2,2019-10-29,면목동,용마산로86길,면목동2차미소지움아파트,12,78.56,2005,45400,중랑구,면목동,면목동2차미소지움아파트,127.094170,37.589641,망우역[경의중앙선],25.379852
3,2019-12-04,면목동,용마산로86길,면목동2차미소지움아파트,4,81.96,2005,45500,중랑구,면목동,면목동2차미소지움아파트,127.094170,37.589641,망우역[경의중앙선],25.379852
4,2019-12-26,면목동,용마산로86길,면목동2차미소지움아파트,3,82.28,2005,45500,중랑구,면목동,면목동2차미소지움아파트,127.094170,37.589641,망우역[경의중앙선],25.379852


데이터 요약이 포함되는 데이터 병합

In [18]:
demo_df = pd.read_csv("고객별_인구통계정보.csv", engine = "python")
purchasing_df = pd.read_csv("고객별_구매금액.csv", engine = 'python')

purchasing_aggregated_df = purchasing_df.groupby('고객ID')['구매금액'].sum()
purchasing_aggregated_df.rename('구매금액합계')

고객ID
고객1     2494480
고객10    2644200
고객2     2573880
고객3     2998590
고객4     2875420
고객5     2628430
고객6     2419410
고객7     2709620
고객8     2914460
고객9     2283150
Name: 구매금액합계, dtype: int64

In [19]:
demo_df.head()

,고객ID,나이,성별,주소
0,고객1,48,남,A아파트
1,고객2,40,남,B아파트
2,고객3,37,여,C아파트
3,고객4,35,여,A연립주택
4,고객5,42,남,B연립주택


In [20]:
merged_df = pd.merge(demo_df, purchasing_aggregated_df, left_on = '고객ID', right_index = True)
merged_df.head()

,고객ID,나이,성별,주소,구매금액
0,고객1,48,남,A아파트,2494480
1,고객2,40,남,B아파트,2573880
2,고객3,37,여,C아파트,2998590
3,고객4,35,여,A연립주택,2875420
4,고객5,42,남,B연립주택,2628430
